In [5]:
pip install boto3==1.34.66

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 81.0 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.32
    Uninstalling botocore-1.34.32:
      Successfully uninstalled botocore-1.34.32
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.32
    Uninstalling boto3-1.34.32:
      Successfully uninstalled boto3-1.34.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.32 requires botocore==1.34.32, but you have botocore 1.34.67 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
boto3.__version__

'1.34.66'

In [339]:
import boto3
import json
import base64
import time
import re
import os
from botocore.exceptions import ClientError
import logging
import redis
from datetime import datetime , date
from insights import construct_call_conversation , enrollment_prompt_generator , load_claude2 , sns_data_postprocessing  
from insights import sns_publisher , get_insights_prompt , get_enrollment_prompt , insights_prompt_generator, redis
import insights_prompt_library

today_date = date.today()

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)
 
bedrock = boto3.client(
    service_name='bedrock', 
    region_name= os.environ.get("Region")  #'us-east-1'
)
 
bedrock_agent_runtime = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)

#prompts config
with open("prompts_config.json") as file:
    prompts_config = json.load(file)

# Initialize Redis client

redis_host = 'ch-agent-assist-redis-cluster-disabled.tjyhst.ng.0001.use1.cache.amazonaws.com'
#os.environ.get("RedisReadUrl") # 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'

redis_port = 6379 # Default Redis port is 6379

redis_password = None  # None if no password

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
 
prompt_bucket = 'ch-agent-assist-prompt-library-bucket'

file_key = 'prompts_library.json'

#AWS SDK for importing the KB
def retrieveAndGenerate(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
                }
            }
        )

def retrieveAndGenerate_new(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'HZ2VTAKFFD',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            }
        )


def retrieveAndGenerate_insurance_pharmacy(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            }
        )

#Checking response
#kb_response = retrieveAndGenerate("What are the different type of insurance types", "WWGXWYHCZE")["output"]["text"]
#print(kb_response)
#kb_response_payer_pharmacy_list = retrieveAndGenerate("What are the different type of insurance providers? and for each of insurance providers, provide the time to fill for all of the specialty pharamcies, provide the response in a structure JSON format", "WWGXWYHCZE")["output"]["text"]
#print(kb_response_payer_pharmacy_list) 

#Passing the payer vs pharmacy stats as part of the context
 
entity_dict_v1 = {
    "patientFirstName": "",
    "patientMiddleInitial": "",
    "patientLastName": "",
    "dob": "",
    "age": "",
    "gender": "",
    "preferredLanguage": "",
    "streetAddress": "",
    "streetName": "",
    "city": "",
    "state": "",
    "zipCode": "",
    "email": "",
    "phone": "",
    "phoneType": "",
    "priorTherapy": "",
    "diagnosisName": "",
    "diagnosisIcd10Code": "",
    "dateOfDiagnosis": "",
    "insuranceType": "",
    "insuranceProvider": "",
    "insuranceId": "",
    "planType": "",
    "effectiveDate": "",
    "expiryDate": "",
    "recordType": "",
    "rxBin": "",
    "rxGroup": "",
    "rxPcn": "",
    "cardHolderRelationshipWithThePatient": "",
    "cardHolderName": "",
    "cardHolderDob": "",
    "prescriberName": "",
    "specialty": "",
    "address": "",
    "facilityName": "",
    "pharmacyName": "",
    "spPhone": "",
    "spFax": ""
}

entity_list = list(entity_dict_v1.keys())

entities = ""

for val in entity_list:
    entities += val + "," 

entities = entities[:len(entities)-1]
# Lambda handler to intgerate with AWS

def handler(event,context):
    start_timestamp = datetime.now()
    print(f'insights_data: handler: START: {datetime.now()}: {str(event)}')
    contact_id = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    #print(key_to_read)
    # Check if the key exists and read the value from Redis
    if contact_id and redis_client.exists(contact_id):
        value = redis_client.get(contact_id)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
    #final_transcript = ""
    #for i in range(len(event['Records'])):
     #   final_transcript += "\n" + data_preprocessing(event['Records'][i])
    final_transcript = construct_call_conversation(value)
    #print(final_transcript)
    
    #sample_transcript = "Customer : Hello , I'm Mark."
    #prompt_enrollment = get_enrollment_prompt(prompt_bucket,file_key,'entity_extraction','enrollment_form_claude',final_transcript)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities,today_date)
    #prompt_enrollment = enrollment_prompt_generator(sample_transcript,entities)
    entity_extraction_model_invoke_timestamp = datetime.now()
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    entity_extraction_model_invoke_after_timestamp = datetime.now()
    print(enrollment_data)
    #json_data = data_postprocessing(enrollment_data)
    json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
    match = json_pattern.search(enrollment_data)
    extracted_json = match.group(0)
    enrollment_json_object = json.loads(extracted_json)
    print("Printing all the keys",list(enrollment_json_object.keys()))
    enrollment_keys = list(enrollment_json_object.keys())
    if "insuranceProvider" in enrollment_json_object.keys() and "pharmacyName" in enrollment_json_object.keys() and enrollment_json_object["insuranceProvider"] != "" and enrollment_json_object["pharmacyName"] != "":
        redis_key = str(json.loads(event["Records"][0]['body'])["streamConnectionId"]) + '::insights_data'
        if redis_client.exists(redis_key):
            print(f'redis_key:{redis_key}')
            insights_redis_data_value = redis_client.get(redis_key)
            print(f'insights_redis_data_value:{insights_redis_data_value}')
            insights_redis_data = eval(insights_redis_data_value)
            print(f'insights_redis_data : {insights_redis_data}')
        #redis_client.set(redis_key,str([{"insuranceProvider":'United Health','pharmacyName':'CVS','insights':'Insights'}]))
        #redis_client.set(redis_key,str([]))
        #insights_redis_data_value = redis_client.get(redis_key)
        #print(f'insights_redis_data_value:{insights_redis_data_value}')
        #insights_redis_data = eval(insights_redis_data_value)
        #print(f'insights_redis_data : {insights_redis_data}')
        if len(insights_redis_data) == 0:
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {insights_redis_data}')
            redis_client.set(redis_key,str(insights_redis_data))
            return {'statusCode':200,'body':'Data Sent to SNS Sucessfully.'}
          
        for val in insights_redis_data:
            if val['insuranceProvider'] == enrollment_json_object['insuranceProvider'] and val['pharmacyName'] == enrollment_json_object['pharmacyName']:
                print(f'insights_redis_data:{insights_redis_data}')
                return {"statusCode" : 200 , "body" : "Generated Insight already present in the database."}
                print('Generated Insight already present in the database.')
            #pass
            else:
                print('Have come to else...')
                insurance_provider = enrollment_json_object["insuranceProvider"]
                pharmacyName = enrollment_json_object["pharmacyName"]
                #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
                insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
                insights_model_invoke_timestamp = datetime.now()
                insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
                insights_model_invoke_after_timestamp = datetime.now()
                print("Generating Insights successfully", insights)
                sns_data = sns_data_postprocessing(event,insights)
                print("sns_data : ",sns_data)
                sns_publisher(sns_data)
                insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
                print(f'insights_redis_data:{insights_redis_data}')
                redis_client.set(redis_key,str(insights_redis_data))
                end_timestamp = datetime.now()
                duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
                duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
                duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
                duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
                duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
                print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
                return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
        else:
            new_value_list = []
            redis_client.set(redis_key,str([])) 
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            new_value_list.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {new_value_list}')
            redis_client.set(redis_key,str(new_value_list))
            print('Insights Lambda Triggered.')
            end_timestamp = datetime.now()
            duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
            duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
            duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
            duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
            duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
            print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
            return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
    else:
        #sns_data = sns_data_postprocessing(event," ")
        print("No Insigts Generated.")
        return {"statusCode": 200,"body": "No Insigts Generated."}
    #SQS_Publisher(sqs_data)
    #print("sns_data : ",sns_data)

In [340]:
prompts_config

{'insurance_type_prompt_generator': {'params': ['insurance_type']},
 'insurance_type_and_provider_prompt_generator': {'params': ['insurance_type',
   'insurance_provider']},
 'insurance_type_provider_pharmacy_ttf_prompt_generator': {'params': ['insurance_type',
   'insurance_provider',
   'pharmacy_name']},
 'insurance_type_provider_pharmacy_prompt_generator': {'params': ['insurance_type',
   'insurance_provider',
   'pharmacy_name']}}

### Sandbox Experiment  - Start

In [ ]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_payer_pharmacy_stats(insurance_type,insurance_provider,pharmacy_name):
    prompt = '''Human:
You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.

For a hub patient with insurance_type as  \' ''' + insurance_type + ''' '\ and insurance_provider as  \' ''' + insurance_provider + ''' '\
Recommend the specialty pharmacies with theirTime to Fill days and compare the time to fill days with the patient's specialty pharmacy \' ''' + pharmacy_name + ''' '\

Note: Only consider the cases when the patient is with Hub.

Provide the response in a structured and readable format.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_payer_pharmacy_stats("Commercial", "Humana", "CVS")



#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")

### Sandbox Experiment  - End

In [10]:
bedrock_agent_runtime = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)

def retrieveAndGenerate(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
                }
            }
        )

def retrieveAndGenerate_new(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'HZ2VTAKFFD',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            }
        )


def retrieveAndGenerate_insurance_pharmacy(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            }
        )



### Insights 1 prompt - Based on just the insurance provider and only inlcuding hub patients

In [11]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_insurance_provider(insurance_type):
    prompt = '''Human:
You are Agent assist tracking the Patient and agent conversation and help the agent by generating an insight, based on the patient's data for insurance provider ''' + insurance_type + '''.
Provide the details.

Note: Only consider the cases when the patient is with Hub.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_insurance_provider("Commercial")


For patients with Commercial insurance and initiated through the Hub, the average conversion rate is 79%, the average time to fill is 12 days, and the average percentage of patients staying on treatment for more than a year is 62%.


### Insights 2 prompt - Based on just the insurance provider,payer only inlcuding hub patients

In [12]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_insurance_provider(insurance_type,insurance_provider):
    prompt = '''Human:
You are Agent assist tracking the Patient and agent conversation and help the agent by generating an insight based on the patient's data for insurance provider ''' + insurance_type + ''' and the insurance_provider   ''' + insurance_provider + '''
Provide all the details including details around PA

Note: Only consider the cases when the patient is with Hub.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_insurance_provider("Commercial","Humana")


For patients with commercial insurance through Humana that are initiated through the hub:

- The conversion rate is 79% 
- The time to fill is 12 days
- 50% of patients require prior authorization
- The time to fill when prior authorization is required is 14 days
- The prior authorization approval rate is 82%
- 61% of patients stay on treatment for more than a year if enrolled in a financial assistance program


### Insights 3 prompt - Based on just the insurance provider,payer and pharmacy and only inlcuding hub patients

In [13]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_payer_pharmacy_stats(insurance_type,insurance_provider,pharmacy_name):
    prompt = '''Human:
You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.

For a hub patient with insurance_type as  \' ''' + insurance_type + ''' '\ and insurance_provider as  \' ''' + insurance_provider + ''' '\
Recommend the specialty pharmacies with theirTime to Fill days and compare the time to fill days with the patient's specialty pharmacy \' ''' + pharmacy_name + ''' '\

Note: Only consider the cases when the patient is with Hub.

Provide the response in a structured and readable format.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_payer_pharmacy_stats("Commercial", "Humana", "CVS")



#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")

For a hub patient with insurance type 'Commercial' and insurance provider 'Humana', the specialty pharmacies and their corresponding Time to Fill days are:

Walgreens: 9 days
CVS: 10 days
Elixir: 17 days
Cigna: 13 days

Compared to CVS which has a Time to Fill of 10 days, Walgreens has a lower Time to Fill of 9 days for this patient profile.


### Insights 3 prompt - Based on just the insurance provider,payer and pharmacy and only inlcuding hub patients

In [15]:
mbedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_payer_pharmacy_stats(insurance_type,insurance_provider,pharmacy_name):
    prompt = '''Human:
You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.

For a hub patient with insurance_type as  \' ''' + insurance_type + ''' '\ and insurance_provider as  \' ''' + insurance_provider + ''' '\
and patient's specialty pharmacy \' ''' + pharmacy_name + ''' '\ , provide all the important metrics for the same combination

Note: Only consider the cases when the patient is with Hub.

Provide the response in a structured and readable format.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_payer_pharmacy_stats("Commercial", "Humana", "CVS")



#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")

For a hub patient with insurance type 'Commercial' and insurance provider 'Humana' using CVS as the specialty pharmacy, the relevant metrics are:

Conversion %: 76%
Time To Fill (days): 10
% of patients staying for more than a year: 69%
% of Patients Requiring Prior Authorization (PA): 66%
Time To Fill when PA required (days): 12
PA% Approved: 90%
% of Patients on treatment if used with financial assistance program: 58%


### Insights 3 prompt - Based on just the insurance provider,payer and pharmacy and only inlcuding hub patients

In [16]:
mbedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_payer_pharmacy_stats(insurance_type,insurance_provider,pharmacy_name):
    prompt = '''Human:
You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.

For a hub patient with insurance_type as  \' ''' + insurance_type + ''' '\ and insurance_provider as  \' ''' + insurance_provider + ''' '\
and patient's specialty pharmacy \' ''' + pharmacy_name + ''' '\ , provide a comparative analysis for all the metrics with a pharmacy performing better.
Note: Only consider the cases when the patient is with Hub.

Provide the response in a structured and readable format.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_payer_pharmacy_stats("Commercial", "Humana", "CVS")



#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")

For a hub patient with insurance type 'Commercial' and insurance provider 'Humana' who is currently using CVS as their specialty pharmacy, the following comparative analysis can be provided based on the search results:

Compared to CVS, Walgreens performs better on the following metrics:
- Higher conversion percentage (88% vs 76%)
- Lower time to fill (9 days vs 10 days)
- Higher percentage of patients staying for more than a year (65% vs 66%)
- Lower percentage of patients requiring prior authorization (11% vs 12%) Elixir also performs better than CVS on some metrics:
- Higher conversion percentage (69% vs 76%) 
- Lower time to fill (17 days vs 10 days)
- Lower percentage of patients requiring prior authorization (19% vs 12%)

However, Elixir has a lower percentage of patients staying for more than a year (37% vs 66%) compared to CVS. Cigna performs worse than CVS on most metrics except for a slightly higher percentage of patients staying for more than a year (50% vs 66%).

Therefore,

In [304]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_insurance_payer_stats(insurance_type,insurance_provider):
    prompt = '''Human:
You are Agent assist tracking the Patient and agent conversation and help the agent by generating some insights for the 
patient's insurance provider ''' + insurance_provider + ''', Insurance type ''' + insurance_type + ''', generate meaningful insights
,only consider the hub initiations.

Provide the response in a structured and easily readable format


Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_insurance_payer_stats("Commercial", "Humana")



For patients with Commercial insurance through Humana that were referred through the hub:

- The average conversion rate is 79% 
- The average time to fill is 12 days
- 59% of patients stay on treatment for more than 1 year Compared to the average for all Commercial plans referred through the hub:

- The conversion rate is slightly higher than average (74%)
- The time to fill is the same as average (12 days) 
- The 1 year persistence is slightly lower than average (62%)


In [256]:
def insights_prompt_generator_insurance_type_stats(insurance_type,kb_response_insurance_details):
    prompt_claude_insurance_provider = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent with the  conversion rate and time to fill data.

The patients's insurance type is  \" """ + insurance_type + """ \" and use the following data to provide the insights:
\" """ + kb_response_insurance_details + """ \".

Only provide the response when you have the values for you have the patient's insurance type from the patient.Do not generate any insights unless you have this information.

Provide the response in a concise and structured and easily readable format


Assistant:
"""
    return prompt_claude_insurance_provider



In [257]:
sample_data = 'Customer : My insurance type is Commercial'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_type = enrollment_json_object["insuranceType"]
print("Insurance Type: ", insurance_type)
insights_prompt_for_insurance_provider = insights_prompt_generator_insurance_type_stats(insurance_type,kb_response_insurance_details)
#print("checking prompt output:", insights_prompt_for_insurance_provider)
insights = load_claude2(bedrock_runtime,insights_prompt_for_insurance_provider,0.5,0.9,1)
print("Generated insights: ",insights)

Insurance Type:  Commercial
Generated insights:   I do not have enough information about the patient's insurance type to provide insights on conversion rate and time to fill. Please provide the patient's insurance type before I can give a structured response with the relevant data.


In [307]:
kb_response_payer_pharmacy_ttf= retrieveAndGenerate(f"""What are all the insurane providers and their corresponding specialty pharmacies for the insurance type as Commercial. Provide the repsonse in a structured output """, "WWGXWYHCZE")["output"]["text"]
print("TTF stats:",kb_response_payer_pharmacy_ttf) 


TTF stats: Commercial Express Scripts Hub:
- Walgreens
- CVS  
- Elixir
- Cigna Commercial Express Scripts SP:
- Walgreens
- CVS
- Elixir
- Cigna Commercial Humana Hub: 
- Walgreens
- CVS
- Elixir  
- Cigna Commercial Humana SP:
- Walgreens Commercial United Health Hub:
- Walgreens
- CVS
- Elixir
- Cigna Commercial United Health SP: 
- Walgreens
- CVS
- Elixir
- Cigna Commercial OptumRX Hub:
- Walgreens
- CVS
- Elixir
- Cigna Commercial OptumRX SP:
- Walgreens 
- CVS
- Elixir
- Cigna


In [328]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

def insights_prompt_generator_payer_pharmacy_stats(insurance_type,insurance_provider,pharmacy_name):
    prompt = '''Human:
You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.

For a hub patient with insurance_type as  \' ''' + insurance_type + ''' '\ and insurance_provider as  \' ''' + insurance_provider + ''' '\
Recommend the specialty pharmacies with theirTime to Fill days and compare the time to fill days with the patient's specialty pharmacy \' ''' + pharmacy_name + ''' '\

Note: Only consider the cases when the patient is with Hub.

Provide the response in a structured and readable format.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

insights_prompt_generator_payer_pharmacy_stats("Commercial", "Humana", "CVS")



#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")

For a hub patient with insurance type 'Commercial' and insurance provider 'Humana', the specialty pharmacies and their corresponding Time to Fill days are:

Walgreens: 9 days
CVS: 10 days
Elixir: 17 days
Cigna: 13 days

Compared to CVS which has a Time to Fill of 10 days, Walgreens has a lower Time to Fill of 9 days for this patient profile.


In [171]:
def insights_prompt_generator_payer_pharmacy_stats(insurance_provider,specialty_pharmacy,kb_response_payer_pharmacy_ttf):
    prompt_claude_payer_pharmacy_stats = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which specialty pharmacy to select based on the
patient's payer and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's insurance provider is  \" """ + insurance_provider + """ \" and pharmacy is  \" """ + specialty_pharmacy + """ \" and use the following data to provide the insights:
\" """ + kb_response_payer_pharmacy_ttf + """ \".


Only provide the response when you have the values for both payer and pharmacy name from the patient.Do not generate any insights unless you have both these information.
Also do not recommend any insights if the patient already has the pharmacy which is the quickest and just suggest the patient to continue
with their existing pharmacy and not change their choice of pharmacy.


Please Provide the recommendation in brief without missing any important details in about 6 lines

Provide the response in a structured and easily readable format


Assistant:
"""
    return prompt_claude_payer_pharmacy_stats



In [175]:
sample_data = 'Customer : My insuranceProvider is Express Scripts and my preferred Pharmacy is CVS'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
print("payer is: ", insurance_provider)
specialty_pharmacy = enrollment_json_object["pharmacyName"]
print("sp is:", specialty_pharmacy)
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
prompt_claude_payer_pharmacy_stats = insights_prompt_generator_payer_pharmacy_stats(insurance_provider,specialty_pharmacy,kb_response_payer_pharmacy_ttf)
print("checking prompt output:", prompt_claude_payer_pharmacy_stats)
#insights_model_invoke_timestamp = datetime.now()
insights_payer_phar = load_claude2(bedrock_runtime,prompt_claude_payer_pharmacy_stats,0.5,0.9,1)
print("Generated insights: ",insights)

payer is:  Express Scripts
sp is: CVS
checking prompt output: Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which specialty pharmacy to select based on the
patient's payer and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's insurance provider is  " Express Scripts " and pharmacy is  " CVS " and use the following data to provide the insights:
" Insurance Type: Commercial

Hub Providers:
- Express Scripts + Walgreens: TTF 10 days (Source 3)  
- Express Scripts + CVS: TTF 15 days (Source 3)
- Express Scripts + Elixir: TTF 11 days 

In [176]:
sample_data = 'Customer : My insuranceProvider is Humana and my preferred Pharmacy is CVS'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
print("payer is: ", insurance_provider)
specialty_pharmacy = enrollment_json_object["pharmacyName"]
print("sp is:", specialty_pharmacy)
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
prompt_claude_payer_pharmacy_stats = insights_prompt_generator_payer_pharmacy_stats(insurance_provider,specialty_pharmacy,kb_response_payer_pharmacy_ttf)
print("checking prompt output:", prompt_claude_payer_pharmacy_stats)
#insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,prompt_claude_payer_pharmacy_stats,0.5,0.9,1)
print("Generated insights: ",insights)

payer is:  Humana
sp is: CVS
checking prompt output: Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which specialty pharmacy to select based on the
patient's payer and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's insurance provider is  " Humana " and pharmacy is  " CVS " and use the following data to provide the insights:
" Insurance Type: Commercial

Hub Providers:
- Express Scripts + Walgreens: TTF 10 days (Source 3)  
- Express Scripts + CVS: TTF 15 days (Source 3)
- Express Scripts + Elixir: TTF 11 days (Source 3)
- Expre

##  Territory Dataset

In [95]:
kb_response_payer_pharmacy_list = retrieveAndGenerate_new("what are all the different payers and the specialty pharmacies under east territory and Medicare insurance type? Only consisder the Hub initiation sources ", "HZ2VTAKFFD")["output"]["text"]
print("Combination of payer with phramacies are:",kb_response_payer_pharmacy_list) 


Combination of payer with phramacies are: For the East territory and Medicare insurance type, considering only Hub initiation sources, the payers and specialty pharmacies are:

Payer: CMS
Specialty Pharmacies: CVS, Elixir, Senderra Rx, Walgreens

Payer: Humana Medicare Advantage
Specialty Pharmacies: Accredo, BioPlus, CVS, Elixir, Senderra Rx, Walgreens


In [ ]:
sample_data = 'Customer : My insuranceProvider is CMS and my preferred Pharmacy is Elixir'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
payer = enrollment_json_object["insuranceProvider"]
print("payer is: ", payer)
specialty_pharmacy = enrollment_json_object["pharmacyName"]
print("sp is:", specialty_pharmacy)
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt_payer_pharmacy = insights_prompt_generator_payer_pharmacy(payer,specialty_pharmacy,kb_response_payer_pharmacy_TTF)
print("checking prompt output:", insights_prompt_payer_pharmacy)
#insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt_payer_pharmacy,0.5,0.9,1)
print("Generated insights: ",insights)

In [120]:
kb_response_payer_pharmacy_TTF = retrieveAndGenerate_new(f""" Get the time to fill(TTF) for these payers and specialty pharmacies, Only consisder the Hub initiation sources,provide the payer name as it is, do not create any information on your own, provide the output in a structured json format  {kb_response_payer_pharmacy_list}""", "HZ2VTAKFFD")["output"]["text"]
print("SPs with TTF:",kb_response_payer_pharmacy_TTF) 

SPs with TTF: Here is the time to fill (TTF) for the specified payers and specialty pharmacies in the East territory and Medicare insurance type, considering only Hub initiation sources, in a structured JSON format:

{
  "CMS": {
    "CVS": 9,
    "Elixir": 10,
    "Senderra Rx": 10,
    "Walgreens": 10
  },
  "Humana Medicare Advantage": {
    "Accredo": 9,
    "BioPlus": 6,
    "CVS": 7,
    "Elixir": 6,
    "Senderra Rx": 6,
    "Walgreens": 6
  }
}


In [128]:
def insights_prompt_generator_payer_pharmacy(payer,specialty_pharmacy,kb_response_payer_pharmacy_TTF):
    prompt_claude_payer_pharmacy = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which specialty pharmacy to select based on the
patient's payer and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's payer is  \" """ + payer + """ \" and pharmacy is  \" """ + specialty_pharmacy + """ \" and use the following data to provide the insights:
\" """ + kb_response_payer_pharmacy_TTF + """ \".


Only provide the response when you have the values for both payer and pharmacy name from the patient.Do not generate any insights unless you have both these information.
Also do not recommend any insights if the patient already has the pharmacy which is the quickest and just suggest the patient to continue
with their existing pharmacy and not change their choice of pharmacy.


Please Provide the recommendation in brief without missing any important details in about 6 lines

Provide the response in a structured and easily readable format


Assistant:
"""
    return prompt_claude_payer_pharmacy



In [130]:
sample_data = 'Customer : My insuranceProvider is CMS and my preferred Pharmacy is Elixir'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
payer = enrollment_json_object["insuranceProvider"]
print("payer is: ", payer)
specialty_pharmacy = enrollment_json_object["pharmacyName"]
print("sp is:", specialty_pharmacy)
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt_payer_pharmacy = insights_prompt_generator_payer_pharmacy(payer,specialty_pharmacy,kb_response_payer_pharmacy_TTF)
print("checking prompt output:", insights_prompt_payer_pharmacy)
#insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt_payer_pharmacy,0.5,0.9,1)
print("Generated insights: ",insights)


payer is:  CMS
sp is: Elixir
checking prompt output: Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which specialty pharmacy to select based on the
patient's payer and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's payer is  " CMS " and pharmacy is  " Elixir " and use the following data to provide the insights:
" Here is the time to fill (TTF) for the specified payers and specialty pharmacies in the East territory and Medicare insurance type, considering only Hub initiation sources, in a structured JSON format:

{
  "CMS": {
  

In [52]:
def insights_prompt_generator_payer_pharmacy(insurance_provider,pharmacy_name,kb_response_payer_pharmacy_list):
    prompt_claude_payer_pharmacy = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which pharmacy to select based on the
patient's Insurance provider and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

If \" """ + insurance_provider + """ \"  or \" """ + pharmacy_name + """ \" is not present in \" """ + kb_response_payer_pharmacy_list + """ \" , then just provide a response telling that you don't have sufficient information to provide any insights and do not follow any of the information in rest of this prompt.

The patients's insurance provider is  \" """ + insurance_provider + """ \" and pharmacy is  \" """ + pharmacy_name + """ \" and use the following insurance statistics data to provide the insights:
\" """ + kb_response_payer_pharmacy_list + """ \".


Only provide the response when you have the values for both insurance provider and pharmacy name from the patient.Do not generate any insights unless you have both these information.
Also do not recommend any insights if the patient already has the pharmacy which is the quickest according to the insurance statistics and just suggest the patient to continue
with their existing pharmacy and not change their choice of pharmacy.


Please Provide the recommendation in brief without missing any important details in about 6 lines in less than 1000 characters

Provide the response in a structured and easily readable format


Assistant:
"""
    return prompt_claude_payer_pharmacy



In [53]:
#kb_response_payer_pharmacy_list = retrieveAndGenerate("What are the different insurance providers? provide the average time to fill for all of the specialty pharamcies for the insurance providers", "WWGXWYHCZE")["output"]["text"]
print("Combination of payer with phramacies are:",kb_response_payer_pharmacy_list) 
sample_data = 'Customer : My insuranceProvider is Humana and my preferred Pharmacy is CVS'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
print(insurance_provider)
pharmacyName = enrollment_json_object["pharmacyName"]
print(pharmacyName)
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt_payer_pharmacy = insights_prompt_generator_payer_pharmacy(insurance_provider,pharmacyName,kb_response_payer_pharmacy_list)
#insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt_payer_pharmacy,0.5,0.9,1)
print("Generated insights: ",insights)


Combination of payer with phramacies are: The insurance providers mentioned are:
- Express Scripts
- Humana 
- United Health
- OptumRx
- Cigna The average time to fill for the specialty pharmacies per insurance provider is:

Express Scripts:
- Walgreens: 13 days 
- CVS: 16 days
- Elixir: 15 days
- Cigna: 16 days

Humana: 
- Walgreens: 13 days
- CVS: 15 days 
- Elixir: 15 days
- Cigna: 14 days  

United Health:
- Walgreens: 11 days
- CVS: 15 days
- Elixir: 17 days 
- Cigna: 13 days

OptumRx:  
- Walgreens: 14 days
- CVS: 16 days
- Elixir: 16 days
- Cigna: 15 days 

Cigna:
- No specialty pharmacy time to fill data
Humana
CVS
Generated insights:   Based on the information provided, the patient's insurance provider is Humana and their pharmacy is CVS. According to the insurance statistics data, for Humana insurance, CVS has an average time to fill of 15 days. This is longer than the 13 days for Walgreens and same as the 15 days for Elixir. 

Since the patient already has their prescription

# Prompt Testing

In [15]:
def insights_prompt_generator(kb_response):
    prompt_claude = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance details.Can you tell what all type of insurance types are there as present in the kb  \" """ + kb_response + """ \" 


Assistant:
"""
    return prompt_claude

In [16]:
print(kb_response)
insights_prompt = insights_prompt_generator(kb_response)
print(insights_prompt)

The different types of insurance mentioned in the search results are: - Commercial - Government - Medicaid - Medicare
Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance details.Can you tell what all type of insurance types are there as present in the kb  " The different types of insurance mentioned in the search results are: - Commercial - Government - Medicaid - Medicare " 


Assistant:



In [46]:
sample_data_1 = 'Customer : My Insurance is with Green Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_1,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

 Unfortunately I do not have sufficient information about the patient's insurance provider and pharmacy to provide meaningful insights for this case. Let's move our conversation forward and I'll be happy to assist if more details become available.


In [18]:
sample_data_2 = 'Customer : My Insurance is with ABC Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_2,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

 I do not have sufficient information in the provided insurance statistics data to give any meaningful insights for this patient. The patient's insurance provider "ABC Health" and pharmacy "CVS" are not present in the data. Without having the dispense time metrics for this specific insurance provider and pharmacy combination, I cannot make an informed recommendation. I suggest continuing the conversation with the patient to gather the required details before providing pharmacy recommendations.


In [1]:
sample_data_3 = 'Customer : My Insurance is with United Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_3,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

NameError: name 'enrollment_prompt_generator' is not defined

# Passing prompt directly in Ret and Gen of Kb

In [20]:
insuranceProvider='Humana'
pharmacyName='CVS'

def insights_prompt_generator_payer_pharmacy(insurance_provider,pharmacy_name):
    prompt_claude_payer_pharmacy = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which pharmacy to select based on the
patient's Insurance provider and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's insurance provider is  Humana and pharmacy is  CVS.

Assistant:
"""
    return prompt_claude_payer_pharmacy

insights_prompt_payer_pharmacy = insights_prompt_generator_payer_pharmacy(insuranceProvider,pharmacyName)

insights_from_kb = retrieveAndGenerate(insights_prompt_payer_pharmacy, "WWGXWYHCZE")["output"]["text"]
print("passing prompt directly to the kb, insights generated: ",insights_from_kb)